**NLP КУРСОВОЙ ПРОЕКТ**

**Задание**

#### Реализовать сценарии поведения для диалоговой системы
- 1-ый сценарий болталка:
медицинская вопросно-ответную систему, датасет https://huggingface.co/datasets/blinoff/medical_qa_ru_data
- 2-ой погода в городе
- 3-тий праздники
- 4-ый фильмы
- 5-ый покупка товара

In [62]:
!pip3 install transformers torch  torchvision torchaudio

In [63]:
!pip3 install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

ERROR: Could not find a version that satisfies the requirement torch==1.12.1 (from versions: 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.12.1


In [64]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)


In [73]:
def generate_response(question, max_response_length=200):
    input_text = "Вопрос:" + question  # Префикс "вопрос:" для указания модели на вопрос
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Генерация ответа
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_response_length, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    print("Сгенерированный ответ:", response)
    return response

In [74]:
# Пример использования
user_question = "Какие блюда готовят на Рош-А-Шана?"
answer = generate_response(user_question)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Сгенерированный ответ: Вопрос:Какие блюда готовят на Рош-А-Шана?

ОТВЕТ:
Рош А-Шаны готовят из мяса, рыбы, овощей, зелени, фруктов, орехов, семян, меда, пряностей, специй, а также из яиц, молока, яиц и молочных продуктов.
В Роше-шанах можно найти блюда из риса, бобов, картофеля, моркови, свеклы, лука, чеснока, петрушки, сельдерея, укропа, кинзы, базилика, зеленого лука и других растений. В Роша-шане можно также найти различные блюда, приготовленные из овощей и фруктов. Роши-шан готовят с добавлением молока и яиц.<s>
Праздник к нам приходит... 

С праздником, дорогие друзья!
Пусть в этот день все будет хорошо!<s>	Полезные советы для тех, кто хочет похудеть 
	
1. Не ешьте после шести. Это вредно
Вопрос:Какие блюда готовят на Рош-А-Шана?

ОТВЕТ:
Рош А-Шаны готовят из мяса, рыбы, овощей, зелени, фруктов, орехов, семян, меда, пряностей, специй, а также из яиц, молока, яиц и молочных продуктов.
В Роше-шанах можно найти блюда из риса, бобов, картофеля, моркови, свеклы, лука, чеснока, петрушки

In [67]:
# Загрузка медицинских данных из CSV файла
data = pd.read_csv("medical_qa_ru_data.csv")

In [68]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch


# Функция для получения ответа на медицинский вопрос
def generate_medical_response(question, max_response_length=200):
    input_text = "вопрос:" + question  # Префикс "вопрос:" для указания модели на вопрос
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Генерация ответа
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_response_length, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Остановка генерации перед вторым словом "Вопрос"
    response = response.split("Вопрос:", 1)[0]
    
    print("\n\nСгенерированный ответ:", response)
    return response



# Функция для выбора ближайших соседей на основе TF-IDF и косинусной метрики
def choose_best_answer(question, dataset, num_neighbors=1):
    # Токенизация вопроса пользователя
    question_tokens = tokenizer.tokenize(question)
    question_text = " ".join(question_tokens)

    # Предварительная обработка текста в "desc" из датасета
    dataset_tokens = [tokenizer.tokenize(desc) if isinstance(desc, str) else [] for desc in dataset]
    dataset_text = [" ".join(tokens) for tokens in dataset_tokens]

    # Вычисление TF-IDF векторов для текстов в "desc"
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(dataset_text)

    # Вычисление TF-IDF вектора для вопроса пользователя
    question_tfidf = vectorizer.transform([question_text])

    # Вычисление косинусных сходств между вопросом пользователя и текстами в "desc"
    cosine_similarities = cosine_similarity(question_tfidf, tfidf_matrix)

    # Пороговое значение метрики сходства
    threshold_similarity = 0.5  # Настройте это значение по вашему усмотрению    
    
    # Выбор ближайших соседей из "desc"
    best_indices = np.argsort(cosine_similarities, axis=1)[:, -num_neighbors:][:, ::-1]
    best_descs = [[dataset[idx] for idx in indices] for indices in best_indices]

    # Собираем ответы на основе "ans" для каждого текста из "best_descs"
    answers = []
    for desc_list in best_descs:
        answer_list = []
        for desc in desc_list:
            matching_rows = data[data["desc"] == desc]
            if not matching_rows.empty and "ans" in matching_rows.columns:
                answer = matching_rows["ans"].values[0]
                answer_list.append(answer)
        answers.append(answer_list)

    # Пример: выводим ответы
    for i, answer_list in enumerate(answers):
        print(f"Ответ на основе ближайших соседей:\n")
        for j, answer in enumerate(answer_list):
            print(f"{j + 1}. {answer}")

    return best_descs


In [69]:
# Пример использования
user_question = "Ребенок плачет, что делать? "

# Выбор ответов на основе ближайших соседей
best_descs = choose_best_answer(user_question, data["desc"])

# Если нет близких соседей, использовать generate_medical_response
answer = generate_medical_response(user_question)


/opt/homebrew/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ответ на основе ближайших соседей:

1. Приглашайте педиатра на дом.;
Покажите ребенка педиатру.;
Необходимо показать ребенка врачу.


Сгенерированный ответ: вопрос:Ребенок плачет, что делать? 
Ответ:
Рекомендую вам обратиться к психологу.




# Bot

In [70]:
!pip install pytelegrambotapi

In [81]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import dialogflow

ImportError: cannot import name 'Filters' from 'telegram.ext' (/opt/homebrew/lib/python3.11/site-packages/telegram/ext/__init__.py)

In [75]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text

    update.message.reply_text('Ваше сообщение! ' + update.message.text)
    reply_text = model_ru([text], [update.message.text])

    #.encode('utf8')
    reply_text = reply_text[0][0] 
    
    update.message.reply_text(reply_text)

NameError: name 'Update' is not defined

In [76]:
updater = Updater(my_token, use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

NameError: name 'Updater' is not defined